In [1]:
!pip install config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

import config

In [3]:
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

In [4]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=config.DEVELOPER_KEY)
search_response = youtube.search().list(
    q = '뽀구미',
    order = 'relevance',
    part = 'snippet',
    maxResults = 100
).execute()

In [5]:
channel_id = search_response['items'][0]['id']['channelId']

In [6]:
channel_id

'UCO78BQwAIG-pEfhgJEt-VRQ'

In [7]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=config.DEVELOPER_KEY)
playlists = youtube.playlists().list(
    channelId = channel_id,
    part = 'snippet',
    maxResults = 50
).execute()

In [8]:
import pandas as pd

df = pd.DataFrame()
for item in playlists['items']:
    item_dict = {}
    item_dict['playlist'] = item['snippet']['title']
    item_dict['playlist_id'] = item['id']
    df = df.append(item_dict, ignore_index=True)

In [9]:
df

,playlist,playlist_id
0,뽀구미 화참기 챌린지,PLOgHg9Yl9tMYK1bn6d6y4wbMKtCwovXPq
1,뽀구미 Q&A,PLOgHg9Yl9tMZJGZzjoe06EY8CA6gWnmLQ
2,뽀구미 입덕직캠,PLOgHg9Yl9tMZnkoJp9eFZ3n71JQ0jVpe1
3,뽀구미 뇌절토론 (폐지),PLOgHg9Yl9tMb6e81Ug6_GP0Ty0J2uS-6l
4,뽀구미 이상형월드컵,PLOgHg9Yl9tMaUNlcORlht0s9yitW-4Arl
5,뽀구미 ASMR,PLOgHg9Yl9tMaiTd10eaNyLsDXMvnkXzY2
6,뽀구미 메이크업,PLOgHg9Yl9tMZYHXyf7ANwvoREcYwwuzQV
7,뽀구미 시청자 참여 컨텐츠,PLOgHg9Yl9tMZR3RSTNrfARfDBEpFMNTTX
8,뽀구미 Shorts,PLOgHg9Yl9tMY8oAaSGLEI53DL7Ucdzmy1
9,뽀구미 합방,PLOgHg9Yl9tMaiG8w4jFbft5sRD9I0m0WH


In [10]:
video_ids = []
video_titles = []

for id in df['playlist_id']:
    playlistitems_list_request = youtube.playlistItems().list(
        playlistId = id,
        part = 'snippet',
        maxResults = 50
    )

    while playlistitems_list_request:
        playlistitems_list_response = playlistitems_list_request.execute()

        for item in playlistitems_list_response['items']:
            title = item['snippet']['title']
            video_id = item['snippet']['resourceId']['videoId']

            video_titles.append(title)
            video_ids.append(video_id)

        playlistitems_list_request = youtube.playlistItems().list_next(
            playlistitems_list_request, playlistitems_list_response
        )

video_df = pd.DataFrame()
video_df['video_title'] = video_titles
video_df['video_id'] = video_ids

In [11]:
video_df

,video_title,video_id
0,뽀구미의 화참기 챌린지! 성공하면 문상지급,rsLB7ee1-jA
1,제대로된 질문 좀 하라고 [구독자 7만명 달성 Q&A],aYzJTvi6VF8
2,제대로된 질문이 하나도 없는 [구독자 1만명 달성 Q&A],HgtIID4H9aE
3,뽀구미 코카인 댄스 교차편집,BzcrBlUSHAo
4,누가 밥먹을때 시골집 문닫히는 소리내냐고ㅋㅋ 아 ㄹㅇㅋㅋ,C83EeYfiup4
...,...,...
387,Private video,SsWrG0IDJQE
388,Private video,rRJSxFA2zRg
389,야방중 제로투 댄스 추는데 지나가던 사람과 눈마주쳤습니다ㅋㅋㅋㅋㅋ,7r2fAsViHv0
390,한시라도 맘편히 있을 수 없는 방청소 방송,qFRvrB2bwvw


---

In [12]:
import re

video_ids = []
titles = []
published_date = []
category_ids = []
view_count = []
like_count = []
comment_count = []
tags = []
hours = []
mins = []
secs = []
desc = []

for i in range(len(video_df)):
    request = youtube.videos().list(
        id = video_df['video_id'][i],
        part = 'snippet,contentDetails,statistics'
    )

    response = request.execute()

    if response['items'] == []:
        video_ids.append('-')
        titles.append('-')
        published_date.append('-')
        category_ids.append('-')
        view_count.append('-')
        like_count.append('-')
        comment_count.append('-')
        tags.append('-')
        hours.append('-')
        mins.append('-')
        secs.append('-')
        desc.append('-')

    else:
        try:
            video_ids.append(video_df['video_id'][i])
            titles.append(response['items'][0]['snippet']['title'])
            published_date.append(response['items'][0]['snippet']['publishedAt'].split('T')[0])
            category_ids.append(response['items'][0]['snippet']['categoryId'])
            view_count.append(response['items'][0]['statistics']['viewCount'])
            like_count.append(response['items'][0]['statistics']['likeCount'])
            comment_count.append(response['items'][0]['statistics']['commentCount'])
            tags.append(response['items'][0]['snippet']['tags'])
            desc.append(response['items'][0]['snippet']['description'])

            duration = re.findall(r'\d+', response['items'][0]['contentDetails']['duration'])
            if len(duration) == 3:
                hours.append(duration[0])
                mins.append(duration[1])
                secs.append(duration[2])
            elif len(duration) == 2:
                hours.append('0')
                mins.append(duration[0])
                secs.append(duration[1])
            else:
                hours.append('0')
                mins.append('0')
                secs.append(duration[0])
        
        except:
            pass

data = pd.DataFrame([video_ids, titles, published_date, category_ids, view_count, like_count, comment_count, tags, hours, mins, secs, desc]).T
data.columns = ['video_id', 'title', 'published_date', 'categiry_id', 'view_count', 'like_count', 'comment_count', 'tags', 'hour', 'min', 'sec', 'description']

In [13]:
data

,video_id,title,published_date,categiry_id,view_count,like_count,comment_count,tags,hour,min,sec,description
0,rsLB7ee1-jA,뽀구미의 화참기 챌린지! 성공하면 문상지급,2021-02-16,22,336530,3929,401,"[뽀구미, 뽀꾸미, 보꾸미, 보구미, 뽀구미 화참기, 화참기, 참기, 웃음참기, 뽀...",0,10,27,제 시청자 좀 혼내주세요ㅜㅠ\n-\n안녕하세요트위치 스트리머 뽀구미 입니당!\n구독...
1,aYzJTvi6VF8,제대로된 질문 좀 하라고 [구독자 7만명 달성 Q&A],2022-01-03,23,553767,9306,1328,"[뽀구미, 뽀꾸미, 보구미, 보꾸미, 뽀구미 큐앤에이, 목주름, 뽀구미 목주름, 뽀...",0,13,48,♥생방송은 트위치에서♥\n♥방송시간 및 중요 공지는 트게더에! (아래링크 참조)♥\...
2,HgtIID4H9aE,제대로된 질문이 하나도 없는 [구독자 1만명 달성 Q&A],2021-05-23,22,677236,7304,454,"[뽀구미, 뽀꾸미, 보구미, 보꾸미, 뽀구미 구독자, 구독자 1만, 1만명 달성, ...",0,5,53,정상적인 질문 좀 해 이 바보탱이들아\n\n♥생방송은 트위치에서♥\n♥방송시간 및 ...
3,BzcrBlUSHAo,뽀구미 코카인 댄스 교차편집,2022-01-21,22,643658,6039,2097,"[뽀구미, 뽀꾸미, 보꾸미, 보구미, 뽀구미 코카인댄스, 코카인댄스, 코카인, 코카...",0,0,53,원본음악: Kokain 2021 - initio [JERIDE REMIX]\n\n♥...
4,C83EeYfiup4,누가 밥먹을때 시골집 문닫히는 소리내냐고ㅋㅋ 아 ㄹㅇㅋㅋ,2021-12-31,23,188039,4432,1353,"[뽀구미, 뽀꾸미, 보꾸미, 보구미, 뽀구미 아웁, 뽀구미 먹방, 아웁, 아웁모음집...",0,2,30,♥생방송은 트위치에서♥\n♥방송시간 및 중요 공지는 트게더에! (아래링크 참조)♥\...
...,...,...,...,...,...,...,...,...,...,...,...,...
387,-,-,-,-,-,None,None,None,None,None,None,None
388,-,-,-,-,-,None,None,None,None,None,None,None
389,7r2fAsViHv0,야방중 제로투 댄스 추는데 지나가던 사람과 눈마주쳤습니다ㅋㅋㅋㅋㅋ,2021-06-16,22,19465,None,None,None,None,None,None,None
390,qFRvrB2bwvw,한시라도 맘편히 있을 수 없는 방청소 방송,2021-06-05,22,403054,None,None,None,None,None,None,None


In [14]:
data.isnull().sum()

video_id           0
title              0
published_date     0
categiry_id        0
view_count         0
like_count         8
comment_count     15
tags              20
hour              20
min               20
sec               20
description       20
dtype: int64

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   video_id        392 non-null    object
 1   title           392 non-null    object
 2   published_date  392 non-null    object
 3   categiry_id     392 non-null    object
 4   view_count      392 non-null    object
 5   like_count      384 non-null    object
 6   comment_count   377 non-null    object
 7   tags            372 non-null    object
 8   hour            372 non-null    object
 9   min             372 non-null    object
 10  sec             372 non-null    object
 11  description     372 non-null    object
dtypes: object(12)
memory usage: 36.9+ KB
